In [1]:
import os

In [2]:
%pwd

'c:\\Users\\MCW\\Documents\\Ajith\\MLOps\\project_02\\Interstate_Traffic_Volume_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\MCW\\Documents\\Ajith\\MLOps\\project_02\\Interstate_Traffic_Volume_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_learning_rate: float
    params_features: int

In [6]:
from trafficVolumePrediction.constants import *
from trafficVolumePrediction.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_learning_rate=self.params.LEARNING_RATE,
            params_features=self.params.FEATURES
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2023-12-29 20:34:38,995: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense, LSTM


In [10]:
data = pd.read_csv('artifacts/data_ingestion/data_trafficVolume.csv')

In [11]:
data.head()

,Unnamed: 0,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume,date,time,year,month,day,hour
0,0,0,288.28,0.0,0.0,40,0,0,5545,2012-10-02,09:00:00,2012,10,2,9
1,1,0,289.36,0.0,0.0,75,0,1,4516,2012-10-02,10:00:00,2012,10,2,10
2,2,0,289.58,0.0,0.0,90,0,2,4767,2012-10-02,11:00:00,2012,10,2,11
3,3,0,290.13,0.0,0.0,90,0,2,5026,2012-10-02,12:00:00,2012,10,2,12
4,4,0,291.14,0.0,0.0,75,0,1,4918,2012-10-02,13:00:00,2012,10,2,13


In [12]:
x = np.array(data[["holiday","temp","rain_1h","snow_1h","clouds_all","weather_main","weather_description","year","month","day","hour"]])
y = np.array(data[["traffic_volume"]])
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.20, 
                                                random_state=33)

In [13]:
xtrain.shape

(38563, 11)

[2023-12-29 20:08:11,551: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.
]
[2023-12-29 20:08:12,484: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\optimizers\__init__.py:309: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.
]
Epoch 1/10
[2023-12-29 20:08:13,547: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
 5137/38563 [==>...........................] - ETA: 3:11 - loss: 4882230.5000

KeyboardInterrupt: 

In [15]:
# model = Sequential()
# model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
# model.add(LSTM(64, return_sequences=False))
# model.add(Dense(25))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.fit(xtrain, ytrain, batch_size=500, epochs=10)

Epoch 1/10
78/78 [==============================] - 8s 53ms/step - loss: 14417555.0000
Epoch 2/10
78/78 [==============================] - 4s 49ms/step - loss: 13952990.0000
Epoch 3/10
78/78 [==============================] - 4s 49ms/step - loss: 13246943.0000
Epoch 4/10
78/78 [==============================] - 4s 55ms/step - loss: 12303838.0000
Epoch 5/10
78/78 [==============================] - 14s 178ms/step - loss: 11194548.0000
Epoch 6/10
78/78 [==============================] - 12s 153ms/step - loss: 9995228.0000
Epoch 7/10
78/78 [==============================] - 11s 146ms/step - loss: 8785871.0000
Epoch 8/10
78/78 [==============================] - 10s 131ms/step - loss: 7644311.5000
Epoch 9/10
78/78 [==============================] - 10s 127ms/step - loss: 6632757.0000
Epoch 10/10
78/78 [==============================] - 9s 114ms/step - loss: 5790867.0000


In [12]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        train_model = tf.keras.Sequential()
        train_model.add(tf.keras.layers.LSTM(128, return_sequences=True, input_shape= (self.config.params_features, 1)))
        train_model.add(tf.keras.layers.LSTM(64, return_sequences=False))
        train_model.add(tf.keras.layers.Dense(25))
        self.model = train_model

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, learning_rate):

        prediction = tf.keras.layers.Dense(1)(model.output)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer='adam', loss='mean_squared_error',
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [13]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-12-29 20:35:50,090: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-29 20:35:50,095: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-29 20:35:50,097: INFO: common: created directory at: artifacts]
[2023-12-29 20:35:50,097: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-12-29 20:35:50,369: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.
]
[2023-12-29 20:35:51,879: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-12-29 20:35:52,337: WARNING: module_wrapper: From c:\Users\MCW\.conda\envs\mlops\Lib\site-packages\keras\src\optimizers\__init__.py:309: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.
]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 11, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 11, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                      